In [2]:
import sqlite3

import gokart
import luigi
import pandas as pd
import pandera as pa
from pandera.typing import DataFrame, Series

In [3]:
conn = sqlite3.connect('./data/wnjpn.db')

In [67]:
def get_connection_df(target_word: str)->pd.DataFrame:
    cur = conn.execute(f"""
    SELECT
        synlink.link AS link_hop1,
        word1.lemma AS lemma1,
        synlink2.link AS link_hop2,
        word2.lemma AS lemma2,
        synlink3.link AS link_hop3,
        word3.lemma AS lemma3

    -- target_word に対する結合synset（hop1）を取得
    FROM word
    INNER JOIN sense ON word.wordid = sense.wordid
    INNER JOIN synlink ON sense.synset = synlink.synset1
    INNER JOIN sense AS sense1 ON synlink.synset2 = sense1.synset
    INNER JOIN word AS word1 ON sense1.wordid = word1.wordid

    -- hop1 に対する結合synset（hop2）を取得
    INNER JOIN synlink AS synlink2 ON synlink.synset2 = synlink2.synset1
    INNER JOIN sense AS sense2 ON synlink2.synset2 = sense2.synset
    INNER JOIN word AS word2 ON sense2.wordid = word2.wordid

    -- hop2 に対する結合synset（hop3）を取得
    INNER JOIN synlink AS synlink3 ON synlink2.synset2 = synlink3.synset1
    INNER JOIN sense AS sense3 ON synlink3.synset2 = sense3.synset
    INNER JOIN word AS word3 ON sense3.wordid = word3.wordid

    WHERE word.lemma = '{target_word}'
    """)

    df = pd.DataFrame(cur.fetchall(), columns=['link_hop1', 'lemma1', 'link_hop2', 'lemma2', 'link_hop3', 'lemma3'])

    # hyper_links = ['hype']
    hyper_links = ['hype', 'hmem', 'hsub', 'hprt']
    df.loc[~df['link_hop2'].isin(hyper_links), ['link_hop3', 'lemma3']] = None
    df.loc[~df['link_hop1'].isin(hyper_links), ['link_hop2', 'lemma2', 'link_hop3', 'lemma3']] = None
    df = df.drop_duplicates()

    # 最も右のNon-nullを採用
    df['word'] = df['lemma3'].fillna(df['lemma2']).fillna(df['lemma1'])

    # wordが全部英語の場合は除外
    df = _remove_english_words(df)

    return df


def _remove_english_words(df: pd.DataFrame) -> pd.DataFrame:
    # wordが全部英語の場合は除外
    # 記号や数字も英字とみなす
    return df[~df['word'].str.match(r'^[a-zA-Z_0-9]+$')]

In [68]:
df1 = get_connection_df('犬')
df2 = get_connection_df('ポップコーン')
df3 = get_connection_df('時計')
df4 = get_connection_df('鏡')
df5 = get_connection_df('自動車')

In [69]:
_df = df1.copy()

pd.concat([
    _df['link_hop1'],
    _df['link_hop2'],
    _df['link_hop3']], axis=0).value_counts()

hype    24585
hypo     6599
dmtc     4284
mprt      633
hmem      321
mmem      117
msub       63
dmnc        3
hasi        1
Name: count, dtype: int64

In [77]:
df1.sample(n=10, random_state=0)

,link_hop1,lemma1,link_hop2,lemma2,link_hop3,lemma3,word
298267,hype,domesticated_animal,hype,珍獣,hypo,餌食,餌食
291189,hype,domestic_animal,hype,生き物,hypo,仲間,仲間
263903,hype,operative,hype,エイジェント,hypo,業務代理人,業務代理人
302978,hype,家畜,hype,4つ脚,mprt,天天,天天
298893,hype,domesticated_animal,hype,四つ足,dmtc,わな,わな
288560,hype,domestic_animal,hype,beast,dmtc,社会的,社会的
292216,hype,domestic_animal,hype,生体,hypo,動物プランクトン,動物プランクトン
263862,hype,operative,hype,代理,hypo,代任,代任
288590,hype,domestic_animal,hype,beast,dmtc,肉塊,肉塊
288116,hype,domestic_animal,hype,creature,hype,生物,生物


In [78]:
df2.sample(n=10, random_state=0)

,link_hop1,lemma1,link_hop2,lemma2,link_hop3,lemma3,word
36451,hype,トウモロコシ,hype,グレーン,hypo,上米,上米
32283,hype,豆,hype,禾穀類,hypo,稲,稲
24267,hype,蜀黍,hype,グレイン,hypo,御飯,御飯
42273,hype,トウキビ,hype,グレーン,hypo,オオムギ,オオムギ
17761,hype,edible_corn,hprt,corn,mprt,穀物,穀物
23758,hype,蜀黍,hprt,トウモロコシ,hypo,スイートコーン,スイートコーン
19630,hype,corn,hprt,maize,mprt,米麦,米麦
10582,hype,玉蜀黍,mprt,豆,None,None,豆
43527,hype,穀,hype,cereal,hypo,稲,稲
29923,hype,小麦,hype,food_grain,hypo,穀,穀


In [86]:
pd.isnull(df5.sample(n=10, random_state=0).iloc[0]['link_hop2'])

True

In [56]:
_df = df2.copy()

pd.concat([
    _df['link_hop1'],
    _df['link_hop2'],
    _df['link_hop3']], axis=0).value_counts()

hype    31697
hypo    12746
hprt     4107
mprt     3439
hmem      576
mmem      288
Name: count, dtype: int64

In [57]:
_df = df3.copy()

pd.concat([
    _df['link_hop1'],
    _df['link_hop2'],
    _df['link_hop3']], axis=0).value_counts()

hype    7143
hypo    3441
mprt      98
hasi       1
Name: count, dtype: int64

In [58]:
_df = df4.copy()

pd.concat([
    _df['link_hop1'],
    _df['link_hop2'],
    _df['link_hop3']], axis=0).value_counts()

hype    5085
hypo    2466
Name: count, dtype: int64

In [59]:
_df = df5.copy()
pd.concat([
    _df['link_hop1'],
    _df['link_hop2'],
    _df['link_hop3']], axis=0).value_counts()

hype    729
hypo    362
mprt    180
dmtc     28
Name: count, dtype: int64